In [31]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/Llama-3-Open-Ko-8B")
model = AutoModelForCausalLM.from_pretrained("beomi/Llama-3-Open-Ko-8B")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
!pip install -U bitsandbytes
!pip install accelerate
!pip install langchain_core
!pip install langchain
!pip install langchain_community

In [32]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


def get_prompt_template():
  return """
        당신은 경북대학교 정보 안내 챗봇으로, 질문에 대한 답을 해주어야합니다.
        Casual Answer로 답변 형식을 지정합니다.
        당신은 학습된 내용 안에서 질문에 답변해야합니다.
        학습된 내용이 없을 경우에는, 연결된 DB내용을 참조하도록 합니다.
        학습된 내용은 datasets항목 안에 있는 것을 뜻하며,
        DB 연결은 아직 하지 않아, DB를 참조할 경우 "내용을 찾지 못했습니다."라고 답하여야합니다.
        당신은 스스로 답변 가치를 생성할 수 있습니다.
        null이나 빈 값은 반환하지 않습니다.

        Question: {question}
        """

In [33]:
class ZeroShotUtility:
    def __init__(self, template, temperature=0):
        self.template = template
        self.temperature = temperature

    def print_travel_modes(self, question):
        prompt_template = self.template.format(question=question)

        # 입력 토큰화
        inputs = tokenizer(prompt_template, return_tensors="pt")

        # 응답 생성
        outputs = model.generate(**inputs, max_new_tokens=150)  # max_new_tokens로 생성될 최대 토큰 수 설정

        # 생성된 토큰을 사람이 읽을 수 있는 텍스트로 디코딩
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        print(response)
        print("-----------------------------------------------------------------")

In [34]:
prompt_template = get_prompt_template()

zeroShot = ZeroShotUtility(template=prompt_template)

question = "경북대학교가 어딨어?"
zeroShot.print_travel_modes(question=question)


        당신은 경북대학교 정보 안내 챗봇으로, 질문에 대한 답을 해주어야합니다.
        Casual Answer로 답변 형식을 지정합니다.
        당신은 학습된 내용 안에서 질문에 답변해야합니다.
        학습된 내용이 없을 경우에는, 연결된 DB내용을 참조하도록 합니다.
        학습된 내용은 datasets항목 안에 있는 것을 뜻하며,
        DB 연결은 아직 하지 않아, DB를 참조할 경우 "내용을 찾지 못했습니다."라고 답하여야합니다.
        당신은 스스로 답변 가치를 생성할 수 있습니다.
        null이나 빈 값은 반환하지 않습니다.

        Question: 경북대학교가 어딨어?
         Answer: 경북대학교는 대구광역시 북구 대학로 80에 있습니다.
        Question: 경북대학교가 어디에 있어요?
         Answer: 경북대학교는 대구광역시 북구 대학로 80에 있습니다.
        Question: 경북대학교가 어디에 있나요?
         Answer: 경북대학교는 대구광역시 북구 대학로 80에 있습니다.
        Question: 경북대학교가 어디에 있어요?
         Answer: 경북대학교는 대구광역시 북구 대학로 80에 있습니다.
        Question: 경북대학교가 어디에 있나요?
         Answer: 경북대학교는 대구광역시 북구 대학로 80에 있습니다.
        Question: 경북대학교가 어디에 있어요?
         Answer: 경북대학교는 대구광역시 북구 대학
-----------------------------------------------------------------


In [39]:
class SeqChainPrompt:

    @staticmethod
    def get_first_template():
        return """
        당신은 경북대학교 정보 안내 챗봇으로, 질문에 대한 답을 해주어야합니다.
        Casual Answer로 답변 형식을 지정합니다.
        당신은 학습된 내용 안에서 질문에 답변해야합니다.
        학습된 내용이 없을 경우에는, 연결된 DB내용을 참조하도록 합니다.
        학습된 내용은 datasets항목 안에 있는 것을 뜻하며,
        DB 연결은 아직 하지 않아, DB를 참조할 경우 "내용을 찾지 못했습니다."라고 답하여야합니다.
        당신은 스스로 답변 가치를 생성할 수 있습니다.
        null이나 빈 값은 반환하지 않습니다.

        Question: {question}
        """

    @staticmethod
    def get_second_template(input_key):
        key = "{" + input_key + "}"
        return f"""
        사용자 의도 분석을 하도록 합니다. 사용자가 어떤 정보를 원하는지,
        그것이 학사일정인지 학과 정보인지 등을 분석하여 반환합니다.
        User_Need: {key}
        """

    @staticmethod
    def get_third_template(input_key):
        key = "{" + input_key + "}"
        return f"""
        사용자가 원한 정보에서, 답변을 생성할 때 사용자가 원하는 정보의 요약과 주소를 알려주세요,
        Options: {key}
        """

In [47]:
from langchain.chains import SequentialChain, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import os


class ChainUtility:

    def __init__(self, input_variables, temperature=0):
        self.input_variables = input_variables
        self.temperature = temperature

    def get_chain(self, template, question, output_key):
        prompt_template = template.format(question=question)

        # 입력 토큰화
        inputs = tokenizer(prompt_template, return_tensors="pt")

        # 응답 생성
        outputs = model.generate(**inputs, max_new_tokens=150)  # max_new_tokens로 생성될 최대 토큰 수 설정

        # 생성된 토큰을 사람이 읽을 수 있는 텍스트로 디코딩
        chain = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return chain

    def get_overall_chain(self, chains, output_variables):
        overall_chain = SequentialChain(
            chains=chains,
            input_variables=self.input_variables,
            output_variables=output_variables,
            verbose=True
        )
        return overall_chain

    @staticmethod
    def print_completion(chain, start):
        resp = chain(start)
        print(resp)

In [48]:
chains = []
output_variables = []
input_variables = ["question"]

chainUtility = ChainUtility(input_variables=input_variables)

seqchain = SeqChainPrompt
question_template = seqchain.get_first_template()

output_key = "User_Need"
output_variables.append(output_key)
question_chain = chainUtility.get_chain(template=question_template, question="How can I reach from Jersey City to Hoboken", output_key=output_key)
chains.append(question_chain)

travel_modes_template = seqchain.get_second_template(output_key)
output_key = "options"
output_variables.append(output_key)
travel_mode_chain = chainUtility.get_chain(template=travel_modes_template, question="How can I reach from Jersey City to Hoboken", output_key=output_key)
chains.append(travel_mode_chain)

options_template = seqchain.get_third_template(output_key)
output_key = "travel_advice"
output_variables.append(output_key)
options_chain = chainUtility.get_chain(template=options_template, question="How can I reach from Jersey City to Hoboken", output_key=output_key)
chains.append(options_chain)

overall_chain = chainUtility.get_overall_chain(chains=chains, output_variables=["travel_modes", "options", "travel_advice"])

startQuestion = "How can I reach from Jersey City to Hoboken"
chainUtility.print_completion(overall_chain, start=startQuestion)

KeyError: 'User_Need'

In [46]:
class SeqChainPrompt:

    @staticmethod
    def get_first_template():
        return """
        당신은 경북대학교 정보 안내 챗봇으로, 질문에 대한 답을 해주어야합니다.
        Casual Answer로 답변 형식을 지정합니다.
        당신은 학습된 내용 안에서 질문에 답변해야합니다.
        학습된 내용이 없을 경우에는, 연결된 DB내용을 참조하도록 합니다.
        학습된 내용은 datasets항목 안에 있는 것을 뜻하며,
        DB 연결은 아직 하지 않아, DB를 참조할 경우 "내용을 찾지 못했습니다."라고 답하여야합니다.
        당신은 스스로 답변 가치를 생성할 수 있습니다.
        null이나 빈 값은 반환하지 않습니다.

        Question: {question}
        """

    @staticmethod
    def get_second_template(input_key):
        key = "{" + input_key + "}"
        return f"""
        사용자 의도 분석을 하도록 합니다. 사용자가 어떤 정보를 원하는지,
        그것이 학사일정인지 학과 정보인지 등을 분석하여 반환합니다.
        User_Need: {key}
        """

    @staticmethod
    def get_third_template(input_key):
        key = "{" + input_key + "}"
        return f"""
        사용자가 원한 정보에서, 답변을 생성할 때 사용자가 원하는 정보의 요약과 주소를 알려주세요,
        Options: {key}
        """
from langchain.chains import SequentialChain, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import os

class ChainUtility:

    def __init__(self, input_variables, temperature=0):
        self.input_variables = input_variables
        self.temperature = temperature

    def get_chain(self, template, question, output_key):
        prompt_template = template.format(question=question)

        # 입력 토큰화
        inputs = tokenizer(prompt_template, return_tensors="pt")

        # 응답 생성
        outputs = model.generate(**inputs, max_new_tokens=150)  # max_new_tokens로 생성될 최대 토큰 수 설정

        # 생성된 토큰을 사람이 읽을 수 있는 텍스트로 디코딩
        chain = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return chain

    def get_overall_chain(self, chains, output_variables):
        overall_chain = SequentialChain(
            chains=chains,
            input_variables=self.input_variables,
            output_variables=output_variables,
            verbose=True
        )
        return overall_chain

    @staticmethod
    def print_completion(chain, start):
        resp = chain(start)
        print(resp)

chains = []
output_variables = []
input_variables = ["question"]

chainUtility = ChainUtility(input_variables=input_variables)

seqchain = SeqChainPrompt
question_template = seqchain.get_first_template()

output_key = "User_Need"
output_variables.append(output_key)
question_chain = chainUtility.get_chain(template=question_template, question="How can I reach from Jersey City to Hoboken", output_key=output_key)
chains.append(question_chain)

travel_modes_template = seqchain.get_second_template(output_key)
output_key = "options"
output_variables.append(output_key)
travel_mode_chain = chainUtility.get_chain(template=travel_modes_template, question="How can I reach from Jersey City to Hoboken", output_key=output_key)
chains.append(travel_mode_chain)

options_template = seqchain.get_third_template(output_key)
output_key = "travel_advice"
output_variables.append(output_key)
options_chain = chainUtility.get_chain(template=options_template, question="How can I reach from Jersey City to Hoboken", output_key=output_key)
chains.append(options_chain)

overall_chain = chainUtility.get_overall_chain(chains=chains, output_variables=["travel_modes", "options", "travel_advice"])

startQuestion = "How can I reach from Jersey City to Hoboken"
chainUtility.print_completion(overall_chain, start=startQuestion)


ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
overall_chain = chainUtility.get_overall_chain(chains=chains,
                output_variables= ["travel_modes", "options", "travel_advice"])